In [ ]:
import os
import s3fs

# Add the root project to the path to be able to load from src
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from src.config import RAGConfig

# Default config loaded from default values and environment variables
config = custom_config(
    defaults = { # These defaults can be overriden with env variables
    },
    overrides = { # These values are going to be used no matter what
        "emb_model": "Alibaba-NLP/gte-Qwen2-1.5B-instruct",
        "use_tokenizer_to_chunk": True
    })

filesystem = s3fs.S3FileSystem(endpoint_url=config.s3_endpoint_url)

In [ ]:
from src.db_building.corpus_building import build_or_load_document_database
df, all_splits = build_or_load_document_database(filesystem, config)

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = modelid
embedding_device = "cuda"

emb_model = HuggingFaceEmbeddings(
    model_name=embedding_model,
    model_kwargs={"device": embedding_device},
    encode_kwargs={"normalize_embeddings": True},  # set True for cosine similarity
    show_progress=False,
)

In [ ]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="insee_data",
    embedding_function=emb_model,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [ ]:
ids = [i for i, _ in enumerate(all_splits)]

vector_store.add_documents(documents=all_splits, ids=ids)